# Daten untersuchen mit Pandas und Python für Fortgeschrittene

### Daten aus der CSV-Datei lesen

Nun benutzen wir pandas um die [CSV-Datei](https://de.wikipedia.org/wiki/CSV_(Dateiformat)) einzulesen. Damit das funktioniert muss vorher noch das pandas-Paket mit dem `import`-Befehl unter dem Namen `pd` verfügbar gemacht werden.

Dabei wird ein sogenannter [DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html) mit dem Namen `nba` erzeugt. Vereinfacht kann man sich darunter eine Tabelle mit Spalten und Zeilen vorstellen. DataFrames sind **die** zentralen Datentypen in pandas.   

Mit dem Befehl `type()` kann der Datentyp eines jeden Python-Objekts ausgegeben werden. 

In [ ]:
import pandas as pd
nba = pd.read_csv("nba_all_elo.csv")
type(nba)
# Expected:
# <class 'pandas.core.frame.DataFrame'>

In [ ]:
nba.head()

# Aufgabe: Bulls vs. Pistons

![Bulls](https://upload.wikimedia.org/wikipedia/en/thumb/6/67/Chicago_Bulls_logo.svg/238px-Chicago_Bulls_logo.svg.png) vs ![Pistons](https://upload.wikimedia.org/wikipedia/commons/thumb/7/7c/Pistons_logo17.svg/240px-Pistons_logo17.svg.png)

Im Zeitraum zwischen 1988 und 1991 herrschte ein große Rivalität zwischen den beiden Clubs.

1) Erzeugen Sie einen neuen DataFrame für die folgenden Aufgaben, der nur die benötigten Daten enthält. Ggf. können Sie gerne noch weitere spezialisierte DataFrames anlegen.
2) Wie ist die Bilanz der beiden Teams (Siege und Niederlagen in direkten Duellen) für beide Teams im gesamten Zeitraum?
3) Wie viele Siege erzielten die Bulls in den einzelnen Jahren 88-91?
4) Welches war der höchste Sieg der Bulls im Jahr 1991?
5) Wie viele Punkte pro Jahr erzielten die Bulls im Schnitt in dem genannten Zeitraum. Ermitteln Sie dasselbe für die Pistons. Visualisieren Sie.
6) Visualisieren Sie die Punktedifferenz in den Ergebnissen für alle direkten Duelle mit einem geeigneten Diagramm.
7) Welche Mannschaft hat hat die schlechteste Auswärtsbilanz im Zeitraum? Berechnen Sie den Wert basierend auf Siegen und anschließend basierend auf Punkten. Gezählt werden sollen nur Spiele die nicht in den Playoffs stattfinden. 


## Lösung:

### Teilaufgabe 1

Erzeugen Sie einen neuen DataFrame für die folgenden Aufgaben, der nur die benötigten Daten enthält. Ggf. können Sie gerne noch weitere spezialisierte DataFrames anlegen.

In [ ]:
bullspistons = nba[
    ((nba["fran_id"] == "Bulls") & (nba["opp_fran"] == "Pistons") | # Heimspiel "oder" (Operator: |)
    (nba["fran_id"] == "Pistons") & (nba["opp_fran"] == "Bulls")) & # Auswärtsspiel
    (nba["year_id"] <= 1991) & # Datum bis
    (nba["year_id"] >= 1988) # Datum von
    ]

Das Ergebnis enthält hier zwei Zeilen pro Spiel, einen aus Bulls, einen aus Pistons Perspektive (sieht man sehr gut an der `game_id`).

In [ ]:
bullspistons.head()

### Teilaufgabe 2

Wie ist die Bilanz der beiden Teams (Siege und Niederlagen in direkten Duellen) für beide Teams im gesamten Zeitraum?

In [ ]:
bullspistons.groupby(["fran_id", "game_result"])["game_id"].count()

### Teilaufgabe 3

Wie viele Siege erzielten die Bulls in den einzelnen Jahren 88-91?

In [ ]:
bullspistons[((bullspistons["fran_id"] == "Bulls") & (bullspistons["game_result"] == "W")) 
].groupby(["fran_id", "year_id"])["game_id"].count()

### Teilaufgabe 4

Welches war der höchste Sieg der Bulls im Jahr 1991?

Was heißt der höchste Sieg? a) Absolut oder b) relativ im Vergleich zum Verlierer? 
Wenn b) muss dieser Wert erst einmal entwickelt werden.

In [ ]:
bullspistons["pts_diff"] = bullspistons["pts"]-bullspistons["opp_pts"]

In [ ]:
bullspistons.head()

Gewonnenes Spiel mit der größten Differenz:

In [ ]:
bullspistons[((bullspistons["fran_id"] == "Bulls") & (bullspistons["game_result"] == "W")) 
].pts_diff.max()

Zeilenindex dieses Spiels:

In [ ]:
bullspistons[((bullspistons["fran_id"] == "Bulls") & (bullspistons["game_result"] == "W")) 
].pts_diff.idxmax()

Zugriff auf die ganze Zeile mit `loc()`

In [ ]:
bullspistons.loc[bullspistons[((bullspistons["fran_id"] == "Bulls") & (bullspistons["game_result"] == "W")) 
].pts_diff.idxmax()]

für Alternative a) ist das Vorgehen deutlich einfacher. Größte Punktanzahl bei gewonnen Spielen:

In [ ]:
bullspistons[((bullspistons["fran_id"] == "Bulls") & (bullspistons["game_result"] == "W")) 
].pts.max()

Index dieser Zeile:

In [ ]:
bullspistons[((bullspistons["fran_id"] == "Bulls") & (bullspistons["game_result"] == "W")) 
].pts.idxmax()

Ganze Zeile mit dem höchsten absoluten Gewinn per `loc()`

In [ ]:
bullspistons.loc[bullspistons[((bullspistons["fran_id"] == "Bulls") & (bullspistons["game_result"] == "W")) 
].pts.idxmax()]

### Teilaufgabe 5

Wie viele Punkte pro Jahr erzielten die Bulls im Schnitt in dem genannten Zeitraum. Ermitteln Sie dasselbe für die Pistons. Visualisieren Sie.

In [ ]:
bullspistons[((bullspistons["fran_id"] == "Bulls")) 
].groupby(["year_id"])["pts"].mean()

In [ ]:
bullspistons[((bullspistons["fran_id"] == "Bulls")) 
].groupby(["year_id"])["pts"].mean().plot()

In [ ]:
bullspistons[((bullspistons["fran_id"] == "Pistons")) 
].groupby(["year_id"])["pts"].mean()

In [ ]:
bullspistons[((bullspistons["fran_id"] == "Pistons")) 
].groupby(["year_id"])["pts"].mean().plot()

### Teilaufgabe 6

Visualisieren Sie die Punktedifferenz in den Ergebnissen für alle direkten Duelle mit einem geeigneten Diagramm.

In [ ]:
bullspistons[((bullspistons["fran_id"] == "Bulls")) 
].pts_diff.plot(kind="bar",figsize=(20, 4))

### Teilaufgabe 7

Welche Mannschaft hat hat die schlechteste Auswärtsbilanz im Zeitraum? Berechnen Sie den Wert basierend auf Siegen und anschließend basierend auf Punkten. Gezählt werden sollen nur Spiele die nicht in den Playoffs stattfinden. 

In [ ]:
bullspistons[
    (bullspistons["game_location"] == "A") & 
    (bullspistons["game_result"] == "W") &
    (bullspistons["is_playoffs"] == 0)
].groupby(["fran_id"])["game_id"].count()

In [ ]:
bullspistons[
    (bullspistons["game_location"] == "A") &
    (bullspistons["is_playoffs"] == 0)
].groupby(["fran_id"])["pts"].sum()